En el trabajo de Arturo se menciona la ecuacion de Volterra 

In [1]:
import pickle

In [3]:
control = pickle.load(open("../step_1/Control.pickle", "rb"))
control

{'control': array([ 0.        ,  0.00211264,  0.00211264, ..., -0.00211264,
        -0.00211264,  0.        ]),
 'x': array([0.     , 0.     , 0.03125, ..., 0.96875, 1.     , 1.     ]),
 'y': array([1.     , 0.96875, 1.     , ..., 0.     , 0.03125, 0.     ])}

In [4]:
tau = 0.1
dt = 0.0025

In [6]:
N = int(tau/dt)
N

40

In [125]:
ix = 1
x_ix, y_ix = control["x"][ix], control["y"][ix]
control_ix = control["control"][ix]
control_ix

0.0021126359921771146

In [11]:
t = dt*0

In [12]:
sigma = lambda t: 1

In [15]:
import numpy as np

In [126]:
v_tilde = np.array([
    [0],
    [control_ix]
])

L0 = np.array([
    [1, 0],
    [0, sigma(0)]
])

def M_ts(t, s):
    return np.array([
        [0, 1],
        [sigma(s-t), (sigma(s-t+dt)-sigma(s-t))/dt]
    ])

In [127]:
from scipy.linalg import block_diag

In [128]:
ell_0 = block_diag(*[L0 for _ in range(N+1)])
ell_0

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [129]:
ell_0.shape

(82, 82)

In [130]:
raw_em_N = np.array([
    [M_ts(dt*i, dt*j) if (i==j or j==N) and i!=N else 2*M_ts(dt*i, dt*j) if i<j and i!=N else 0*M_ts(dt*i, dt*j) for j in range(N+1)]
    for i in range(N+1)
])
em_N = np.concatenate(raw_em_N[0], axis=1)
for i in range(1, raw_em_N.shape[0]):
    em_N = np.concatenate([em_N, np.concatenate(raw_em_N[i], axis=1)])
em_N

array([[0., 1., 0., ..., 2., 0., 1.],
       [1., 0., 2., ..., 0., 1., 0.],
       [0., 0., 0., ..., 2., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [131]:
A = ell_0 + (dt/2) * em_N

In [144]:
raw_b = np.array([
    v_tilde
    for _ in range(N+1)
])
b = np.concatenate(raw_b, axis=0).flatten()
b

array([0.        , 0.00211264, 0.        , 0.00211264, 0.        ,
       0.00211264, 0.        , 0.00211264, 0.        , 0.00211264,
       0.        , 0.00211264, 0.        , 0.00211264, 0.        ,
       0.00211264, 0.        , 0.00211264, 0.        , 0.00211264,
       0.        , 0.00211264, 0.        , 0.00211264, 0.        ,
       0.00211264, 0.        , 0.00211264, 0.        , 0.00211264,
       0.        , 0.00211264, 0.        , 0.00211264, 0.        ,
       0.00211264, 0.        , 0.00211264, 0.        , 0.00211264,
       0.        , 0.00211264, 0.        , 0.00211264, 0.        ,
       0.00211264, 0.        , 0.00211264, 0.        , 0.00211264,
       0.        , 0.00211264, 0.        , 0.00211264, 0.        ,
       0.00211264, 0.        , 0.00211264, 0.        , 0.00211264,
       0.        , 0.00211264, 0.        , 0.00211264, 0.        ,
       0.00211264, 0.        , 0.00211264, 0.        , 0.00211264,
       0.        , 0.00211264, 0.        , 0.00211264, 0.     

In [145]:
from scipy.linalg import solve

In [146]:
A.shape

(82, 82)

In [147]:
b.shape

(82,)

In [151]:
theta = solve(A, b)
theta

array([-2.11615992e-04,  2.12320799e-03, -2.06308625e-04,  2.12268558e-03,
       -2.01002547e-04,  2.12217644e-03, -1.95697726e-04,  2.12168057e-03,
       -1.90394128e-04,  2.12119795e-03, -1.85091720e-04,  2.12072860e-03,
       -1.79790468e-04,  2.12027249e-03, -1.74490341e-04,  2.11982964e-03,
       -1.69191304e-04,  2.11940004e-03, -1.63893324e-04,  2.11898368e-03,
       -1.58596369e-04,  2.11858057e-03, -1.53300405e-04,  2.11819070e-03,
       -1.48005399e-04,  2.11781407e-03, -1.42711318e-04,  2.11745067e-03,
       -1.37418129e-04,  2.11710051e-03, -1.32125799e-04,  2.11676358e-03,
       -1.26834294e-04,  2.11643988e-03, -1.21543583e-04,  2.11612941e-03,
       -1.16253631e-04,  2.11583216e-03, -1.10964405e-04,  2.11554814e-03,
       -1.05675873e-04,  2.11527734e-03, -1.00388002e-04,  2.11501976e-03,
       -9.51007581e-05,  2.11477540e-03, -8.98141086e-05,  2.11454426e-03,
       -8.45280203e-05,  2.11432633e-03, -7.92424604e-05,  2.11412161e-03,
       -7.39573957e-05,  